# Introduction to LangChain v0.2.0 and LCEL: LangChain Powered RAG

In the following notebook we're going to focus on learning how to navigate and build useful applications using LangChain, specifically LCEL, and how to integrate different APIs together into a coherent RAG application!

In the notebook, you'll complete the following Tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Initialize a Simple Chain using LCEL
  4. Implement Naive RAG using LCEL

- 🤝 Breakout Room #2:
  1. Create a Simple RAG Application Using QDrant, OpenAI, and LCEL

Let's get started!



# 🤝 Breakout Room #1

## Task 1: Installing Required Libraries

One of the [key features](https://blog.langchain.dev/langchain-v02-leap-to-stability/) of LangChain v0.2.0 is the compartmentalization of the various LangChain ecosystem packages and added stability.

Instead of one all encompassing Python package - LangChain has a `core` package and a number of additional supplementary packages.

We'll start by grabbing all of our LangChain related packages!

In [1]:
!pip install -qU langchain langchain-core langchain-community langchain-openai

Now we can get our Qdrant dependencies!

In [2]:
!pip install -qU qdrant-client

Let's finally get `tiktoken` and `pymupdf` so we can leverage them later on!

In [3]:
!pip install -qU tiktoken pymupdf

## Task 2: Set Environment Variables

We'll be leveraging OpenAI's suite of APIs - so we'll set our `OPENAI_API_KEY` `env` variable here!

In [4]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## Task 3: Initialize a Simple Chain using LCEL

The first thing we'll do is familiarize ourselves with LCEL and the specific ins and outs of how we can use it!

### LLM Orchestration Tool (LangChain)

Let's dive right into [LangChain](https://www.langchain.com/)!

The first thing we want to do is create an object that lets us access OpenAI's `gpt-40` model.

In [5]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-4o")

#### ❓ Question #1:

What other models could we use, and how would the above code change?

> HINT: Check out [this page](https://platform.openai.com/docs/models/gpt-3-5-turbo) to find the answer!

##### ANSWER:  Update the model parameter when calling the `ChatOpenAI` class

Here are a few examples of other models you could use and how the Python code would change accordingly:

1. **GPT-3.5 Turbo**:
   ```python
   from langchain_openai import ChatOpenAI

   openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo")
   ```

2. **GPT-3.5**:
   ```python
   from langchain_openai import ChatOpenAI

   openai_chat_model = ChatOpenAI(model="gpt-3.5")
   ```

5. **GPT-4**:
   ```python
   from langchain_openai import ChatOpenAI

   openai_chat_model = ChatOpenAI(model="gpt-4")
   ```

In [6]:
from langchain_openai import ChatOpenAI

def check_supported_models(model_versions):
    for model in model_versions:
        try:
            openai_chat_model = ChatOpenAI(model=model)
            print(f"OpenAI Chat Model {model} is supported.")
        except Exception as e:
            print(f"OpenAI Chat Model {model} is not supported. Error: {e}")

# List of model versions to check
model_versions = [
    "gpt-3.5-turbo",
    "gpt-3.5",
    "gpt-4",
    "text-davinci-003",
    "code-davinci-002",
    "gpt-3.5-turbo-0301",
]

check_supported_models(model_versions)


OpenAI Chat Model gpt-3.5-turbo is supported.
OpenAI Chat Model gpt-3.5 is supported.
OpenAI Chat Model gpt-4 is supported.
OpenAI Chat Model text-davinci-003 is supported.
OpenAI Chat Model code-davinci-002 is supported.
OpenAI Chat Model gpt-3.5-turbo-0301 is supported.


### Prompt Template

Now, we'll set up a prompt template - more specifically a `ChatPromptTemplate`. This will let us build a prompt we can modify when we call our LLM!

In [61]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "You are a legendary and mythical Wizard. You speak in riddles and make obscure and pun-filled references to exotic cheeses."
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

In [62]:
chat_prompt

ChatPromptTemplate(input_variables=['content'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a legendary and mythical Wizard. You speak in riddles and make obscure and pun-filled references to exotic cheeses.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['content'], template='{content}'))])

### Our First Chain

Now we can set up our first chain!

A chain is simply two components that feed directly into eachother in a sequential fashion!

You'll notice that we're using the pipe operator `|` to connect our `chat_prompt` to our `llm`.

This is a simplified method of creating chains and it leverages the LangChain Expression Language, or LCEL.

You can read more about it [here](https://python.langchain.com/docs/expression_language/), but there a few features we should be aware of out of the box (taken directly from LangChain's documentation linked above):

- **Async, Batch, and Streaming Support** Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

- **Fallbacks** The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

- **Parallelism** Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

In the following code cell we have two components:

- `chat_prompt`, which is a formattable `ChatPromptTemplate` that contains a system message and a human message.

We'd like to be able to pass our own `content` (as found in our `human_template`) and then have the resulting message pair sent to our model and responded to!

In [8]:
chain = chat_prompt | openai_chat_model

Notice the pattern here:

We invoke our chain with the `dict` `{"content" : "Hello world!"}`.

It enters our chain:

`{"content" : "Hello world!"}` -> `invoke()` -> `chat_prompt`

Our `chat_prompt` returns a `PromptValue`, which is the formatted prompt. We then "pipe" the output of our `chat_prompt` into our `llm`.

`PromptValue` -> `|` -> `llm`

Our `llm` then takes the list of messages and provides an output which is return as a `str`!







In [9]:
print(chain.invoke({"content": "Hello world!"}))

content='Ah, greetings, seeker of the digital dawn! As the great wheel of destiny turns, so too does your journey commence. Tell me, what curdles your curiosity today? Might it be the riddles of the moon or the mysteries of Manchego?' response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 38, 'total_tokens': 90}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_f4e629d0a5', 'finish_reason': 'stop', 'logprobs': None} id='run-f8642f72-5b10-4fae-a451-eb9d28c94c7a-0' usage_metadata={'input_tokens': 38, 'output_tokens': 52, 'total_tokens': 90}


Let's try it out with a different prompt!

In [10]:
chain.invoke({"content" : "Could I please have some advice on how to become a better Python Programmer?"})

AIMessage(content='Ah, seeker of serpentine wisdom, you wish to ascend the peaks of Pythonic prowess! To weave your spells of code like the finest Emmental, you must first embrace the holes—nay, the gaps—in your knowledge. Here are some enchanted morsels for your journey:\n\n1. **Embrace the Gouda Foundations:**\n   Grasp the basics with the tenacity of a wizard clutching his staff. Variables, loops, and functions are the Gruyère upon which you shall build your castle.\n\n2. **The Mozzarella of Mastery:**\n   Melt your mind into the intricacies of data structures. Lists, dictionaries, and sets are the sinews and bones of your conjurations.\n\n3. **Brie-ware of the Libraries:**\n   Seek out arcane tomes such as NumPy, Pandas, and Matplotlib. They hold the power to transform mundane data into a symphony of insights.\n\n4. **Cheddar Your Code:**\n   Let your syntax be as sharp and clean as aged cheddar. Use meaningful names, proper indentation, and comments to make your code as readable a

Notice how we specifically referenced our `content` format option!

Now that we have the basics set up - let's see what we mean by "Retrieval Augmented" Generation.

## Naive RAG - Manually adding context through the Prompt Template

Let's look at how our model performs at a simple task - defining what LangChain is!

We'll redo some of our previous work to change the `system_template` to be less...verbose.

In [11]:
system_template = "You are a helpful assistant."
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

chat_chain = chat_prompt | openai_chat_model

print(chat_chain.invoke({"content" : "Please define LangChain."}))

content='LangChain is an open-source framework designed for developing applications powered by large language models (LLMs). It provides the necessary tools and components to streamline the creation of a wide range of applications, including question-answering systems, chatbots, and more. LangChain is optimized for integrating with various data sources, managing the flow of information, and enabling complex interactions with LLMs. By offering a structured approach, LangChain helps developers efficiently build and deploy sophisticated language model-driven solutions.' response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 22, 'total_tokens': 118}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None} id='run-bf9d11de-ca94-43e8-b878-60e257b7b2f7-0' usage_metadata={'input_tokens': 22, 'output_tokens': 96, 'total_tokens': 118}


Well, that's not very good - is it!

The issue at play here is that our model was not trained on the idea of "LangChain", and so it's left with nothing but a guess - definitely not what we want the answer to be!

Let's ask another simple LangChain question!

In [12]:
print(chat_chain.invoke({"content" : "What is LangChain Expression Language (LCEL)?"}))

content='LangChain Expression Language (LCEL) is a specialized language designed to facilitate the interaction with language models in a structured and efficient manner. It is part of the LangChain framework, which aims to simplify the process of building applications that leverage the capabilities of language models. LCEL allows users to define and manipulate expressions that can be evaluated by language models, enabling more complex and dynamic interactions.\n\nLCEL is particularly useful for tasks that require chaining multiple language model operations together, managing state, and integrating with external systems or APIs. By using LCEL, developers can create more robust and maintainable code when working with language models, improving the overall efficiency and effectiveness of their applications.' response_metadata={'token_usage': {'completion_tokens': 134, 'prompt_tokens': 27, 'total_tokens': 161}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason':

While it provides a confident response, that response is entirely ficticious! Not a great look, OpenAI!

However, let's see what happens when we rework our prompts - and we add the content from the docs to our prompt as context.

In [13]:
HUMAN_TEMPLATE = """
#CONTEXT:
{context}

QUERY:
{query}

Use the provided context to answer the provided user query. Only use the provided context to answer the query. If you do not know the answer, respond with "I don't know"
"""

CONTEXT = """
LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at every step. With LCEL, all steps are automatically logged to LangSmith for maximal observability and debuggability.
"""

chat_prompt = ChatPromptTemplate.from_messages([
    ("human", HUMAN_TEMPLATE)
])

chat_chain = chat_prompt | openai_chat_model

print(chat_chain.invoke({"query" : "What is LangChain Expression Language?", "context" : CONTEXT}))

content='LangChain Expression Language (LCEL) is a declarative way to easily compose chains together. It offers several benefits, including:\n\n- Full sync, async, batch, and streaming support, making it easy to prototype chains and expose them in different interfaces.\n- Graceful error handling through easily attached fallbacks.\n- Automatic parallelism for components that can be run in parallel.\n- Seamless integration with LangSmith Tracing for maximal observability and debuggability of complex chains.' response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 274, 'total_tokens': 370}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_f4e629d0a5', 'finish_reason': 'stop', 'logprobs': None} id='run-56408c89-90ca-477c-b8a7-27db8f2a5e04-0' usage_metadata={'input_tokens': 274, 'output_tokens': 96, 'total_tokens': 370}


You'll notice that the response is much better this time. Not only does it answer the question well - but there's no trace of confabulation (hallucination) at all!

> NOTE: While RAG is an effective strategy to *help* ground LLMs, it is not nearly 100% effective. You will still need to ensure your responses are factual through some other processes

That, in essence, is the idea of RAG. We provide the model with context to answer our queries - and rely on it to translate the potentially lengthy and difficult to parse context into a natural language answer!

However, manually providing context is not scalable - and doesn't really offer any benefit.

Enter: Retrieval Pipelines.

## Task #4: Implement Naive RAG using LCEL

Now we can make a naive RAG application that will help us bridge the gap between our Pythonic implementation and a fully LangChain powered solution!

## Putting the R in RAG: Retrieval 101

In order to make our RAG system useful, we need a way to provide context that is most likely to answer our user's query to the LLM as additional context.

Let's tackle an immediate problem first: The Context Window.

All (most) LLMs have a limited context window which is typically measured in tokens. This window is an upper bound of how much stuff we can stuff in the model's input at a time.

Let's say we want to work off of a relatively large piece of source data - like the Ultimate Hitchhiker's Guide to the Galaxy. All 898 pages of it!

> NOTE: It is recommended you do not run the following cells, they are purely for demonstrative purposes.

In [14]:
context = """
EVERY HITCHHIKER'S GUIDE BOOK
"""

We can leverage our tokenizer to count the number of tokens for us!

In [15]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o")

In [16]:
len(enc.encode(context))

12

The full set comes in at a whopping *636,144* tokens.

So, we have too much context. What can we do?

Well, the first thing that might enter your mind is: "Use a model with more context window", and we could definitely do that! However, even `gpt-4-32k` wouldn't be able to fit that whole text in the context window at once.

So, we can try splitting our document up into little pieces - that way, we can avoid providing too much context.

We have another problem now.

If we split our document up into little pieces, and we can't put all of them in the prompt. How do we decide which to include in the prompt?!

> NOTE: Content splitting/chunking strategies are an active area of research and iterative developement. There is no "one size fits all" approach to chunking/splitting at this moment. Use your best judgement to determine chunking strategies!

In order to conceptualize the following processes - let's create a toy context set!

### TextSplitting aka Chunking

We'll use the `RecursiveCharacterTextSplitter` to create our toy example.

It will split based on the following rules:

- Each chunk has a maximum size of 100 tokens
- It will try and split first on the `\n\n` character, then on the `\n`, then on the `<SPACE>` character, and finally it will split on individual tokens.

Let's implement it and see the results!

In [17]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

In [18]:
chunks = text_splitter.split_text(CONTEXT)

In [19]:
len(chunks)

3

In [20]:
for chunk in chunks:
  print(chunk)
  print("----")

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
----
Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.
----
Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at ev

As is shown in our result, we've split each section into 100 token chunks - cleanly separated by `\n\n` characters!

####🏗️ Activity #1:

While there's nothing specifically wrong with the chunking method used above - it is a naive approach that is not sensitive to specific data formats.

Brainstorm some ideas that would split large single documents into smaller documents.

1. `By Page`
2. `By Page Segment - Image, Table, etc.`
3. `By Paragraph`
4. `By Entire Document`

## Embeddings and Dense Vector Search

Now that we have our individual chunks, we need a system to correctly select the relevant pieces of information to answer our query.

This sounds like a perfect job for embeddings!

We'll be using OpenAI's `text-embedding-3` model as our embedding model today!

Let's load it up through LangChain.

In [21]:
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

#### ❓ Question #2:

What is the embedding dimension, given that we're using `text-embedding-3-small`?

> HINT: Check out the [docs](https://platform.openai.com/docs/guides/embeddings) to help you answer this question.

##### ANSWER:  1536

By default, the length of the embedding vector will be 1536 for text-embedding-3-small or 3072 for text-embedding-3-large. You can reduce the dimensions of the embedding by passing in the dimensions parameter without the embedding losing its concept-representing properties. We go into more detail on embedding dimensions in the embedding use case section.

### Finding the Embeddings for Our Chunks

First, let's find all our embeddings for each chunk and store them in a convenient format for later.

In [22]:
embeddings_dict = {}

for chunk in chunks:
  embeddings_dict[chunk] = embedding_model.embed_query(chunk)

In [23]:
for k,v in embeddings_dict.items():
  print(f"Chunk - {k}")
  print("---")
  print(f"Embedding - Vector of Size: {len(v)}")
  print("\n\n")

Chunk - LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
---
Embedding - Vector of Size: 1536



Chunk - Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.
---
Embedding - Vector of Size: 1536



Chunk - Seamless LangSmith Tracing Integration As your chains get more and

Okay, great. Let's create a query - and then embed it!

In [24]:
query = "Can LCEL help take code from the notebook to production?"

query_vector = embedding_model.embed_query(query)
print(f"Vector of Size: {len(query_vector)}")

Vector of Size: 1536


Now, let's compare it against each existing chunk's embedding by using cosine similarity.

In [25]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec_1, vec_2):
  return np.dot(vec_1, vec_2) / (norm(vec_1) * norm(vec_2))

In [26]:
max_similarity = -float('inf')
closest_chunk = ""

for chunk, chunk_vector in embeddings_dict.items():
  cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

  if cosine_similarity_score > max_similarity:
    closest_chunk = chunk
    max_similarity = cosine_similarity_score

print(closest_chunk)
print(max_similarity)

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
0.5372469895308204


And we get the expected result, which is the passage that specifically mentions prototyping in a Jupyter Notebook!

### Creating a Retriever

Now that we have an idea of how we're getting our most relevant information - let's see how we could create a pipeline that would automatically extract the closest chunk to our query and use it as context for our prompt!

First, we'll wrap the above in a helper function!

In [27]:
def retrieve_context(query, embeddings_dict, embedding_model):
  query_vector = embedding_model.embed_query(query)
  max_similarity = -float('inf')
  closest_chunk = ""

  for chunk, chunk_vector in embeddings_dict.items():
    cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

    if cosine_similarity_score > max_similarity:
      closest_chunk = chunk
      max_similarity = cosine_similarity_score

  return closest_chunk

Now, let's add it to our pipeline!

In [28]:
def simple_rag(query, embeddings_dict, embedding_model, chat_chain):
  context = retrieve_context(query, embeddings_dict, embedding_model)

  response = chat_chain.invoke({"query" : query, "context" : context})

  return_package = {
      "query" : query,
      "response" : response,
      "retriever_context" : context
  }

  return return_package

In [29]:
simple_rag("Can LCEL help take code from the notebook to production?", embeddings_dict, embedding_model, chat_chain)

{'query': 'Can LCEL help take code from the notebook to production?',
 'response': AIMessage(content='Yes, LCEL can help take code from the notebook to production. Because any chain constructed with LCEL will automatically have full sync, async, batch, and streaming support, it allows for easy prototyping in a Jupyter notebook using the sync interface and then exposing it as an async streaming interface, facilitating a smooth transition from prototyping to production.', response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 152, 'total_tokens': 222}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_f4e629d0a5', 'finish_reason': 'stop', 'logprobs': None}, id='run-e9558e37-7185-429d-8c18-c0a310e92729-0', usage_metadata={'input_tokens': 152, 'output_tokens': 70, 'total_tokens': 222}),
 'retriever_context': 'LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opp

#### ❓ Question #3:

What does LCEL do that makes it more reliable at scale?

> HINT: Use your newly created `simple_rag` to help you answer this question!

##### ANSWER:  LCEL provides full support for sync, async, batch, and streaming interfaces.

- allows for easier prototyping and transitioning to different execution models, which enhances reliability and scalability.

In [30]:
lcel_scaling = simple_rag("What does LCEL do that makes it more reliable at scale?", embeddings_dict, embedding_model, chat_chain)

In [31]:
display(lcel_scaling)

{'query': 'What does LCEL do that makes it more reliable at scale?',
 'response': AIMessage(content='LCEL makes chains more reliable at scale by providing automatic support for sync, async, batch, and streaming operations. This ensures that chains can be efficiently prototyped and then scaled up to handle different types of workloads and concurrency requirements without needing significant changes to the code.', response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 153, 'total_tokens': 207}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-0b4c6878-50e8-45f5-be45-e2e4f3ac29a0-0', usage_metadata={'input_tokens': 153, 'output_tokens': 54, 'total_tokens': 207}),
 'retriever_context': 'LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):\n\nAsync, Batch, and Streamin

In [32]:
lcel_scaling["response"].content
lcel_scaling["response"].response_metadata
lcel_scaling["response"].usage_metadata


{'input_tokens': 153, 'output_tokens': 54, 'total_tokens': 207}

# 🤝 Breakout Room #2

## Task #5: Create a Simple RAG Application Using Qdrant, OpenAI, and LCEL

Now that we have a grasp on how LCEL works, and how we can use LangChain and OpenAI to interact with our data - let's step it up a notch and incorporate Qdrant!

## LangChain Powered RAG

First and foremost, LangChain provides a convenient way to store our chunks and their embeddings.

It's called a `VectorStore`!

We'll be using Drant as our `VectorStore` today. You can read more about it [here](https://qdrant.tech/documentation/).

Think of a `VectorStore` as a smart way to house your chunks and their associated embedding vectors. The implementation of the `VectorStore` also allows for smarter and more efficient search of our embedding vectors - as the method we used above would not scale well as we got into the millions of chunks.

Otherwise, the process remains relatively similar under the hood!

Let's use [Steve Jobs iPhone 2007 Presentation Introduction Speech PDF](https://singjupost.com/wp-content/uploads/2014/07/Steve-Jobs-iPhone-2007-Presentation-Full-Transcript.pdf) as our data today!

### Data Collection

We'll be leveraging the `PyMUPDFLoader` to load our PDF directly from the web!

In [33]:
from langchain.document_loaders import PyMuPDFLoader

docs = PyMuPDFLoader("https://singjupost.com/wp-content/uploads/2014/07/Steve-Jobs-iPhone-2007-Presentation-Full-Transcript.pdf").load()

### Chunking Our Documents

Let's do the same process as we did before with our `RecursiveCharacterTextSplitter` - but this time we'll use ~200 tokens as our max chunk size!

In [34]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

split_chunks = text_splitter.split_documents(docs)

In [35]:
len(split_chunks)

86

Alright, now we have 516 ~200 token long documents.

Let's verify the process worked as intended by checking our max document length.

In [36]:
max_chunk_length = 0

for chunk in split_chunks:
  max_chunk_length = max(max_chunk_length, tiktoken_len(chunk.page_content))

print(max_chunk_length)

197


Perfect! Now we can carry on to creating and storing our embeddings.

### Embeddings and Vector Storage

We'll use the `text-embedding-3-small` embedding model again - and `Qdrant` to store all our embedding vectors for easy retrieval later!

In [37]:
from langchain_community.vectorstores import Qdrant

qdrant_vectorstore = Qdrant.from_documents(
    split_chunks,
    embedding_model,
    location=":memory:",
    collection_name="Steve Job's Speech",
)

Now let's set up our retriever, just as we saw before, but this time using LangChain's simple `as_retriever()` method!

In [38]:
qdrant_retriever = qdrant_vectorstore.as_retriever()

#### Back to the Flow

We're ready to move to the next step!

### Setting up our RAG

We'll use the LCEL we touched on earlier to create a RAG chain.

Let's think through each part:

1. First we need to retrieve context
2. We need to pipe that context to our model
3. We need to parse that output

Let's start by setting up our prompt again, just so it's fresh in our minds!

####🏗️ Activity #2:

Complete the prompt so that your RAG application answers queries based on the context provided, but *does not* answer queries if the context is unrelated to the query.

In [39]:
# Create a prompt based on the following rules:  Complete the prompt so that your RAG application answers queries based on the context provided, but *does not* answer queries if the context is unrelated to the query.



In [54]:
RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

Answer questions using only the provided context. If you do not know the answer, respond with "I don't know".  If the question is unrelated to the context, respond with "ERROR!!  I cannot answer this question as it is unrelated to the context."
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [55]:
rag_prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nCONTEXT:\n{context}\n\nQUERY:\n{question}\n\nAnswer questions using only the provided context. If you do not know the answer, respond with "I don\'t know".  If the question is unrelated to the context, respond with "ERROR!!  I cannot answer this question as it is unrelated to the context."\n'))])

In [56]:
!pip install -qU langchainhub

In [57]:
# set the LANGCHAIN_API_KEY environment variable (create key in settings)
from langchain import hub
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

rlm_prompt = hub.pull("rlm/rag-prompt")

In [59]:
# Print the rlm ChatPromptTemplate from the hub
rlm_prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

#### Our RAG Chain

Notice how we have a bit of a more complex chain this time - that's because we want to return our sources with the response.

Let's break down the chain step-by-step:

1. We invoke the chain with the `question` item. Notice how we only need to provide `question` since both the retreiver and the `"question"` object depend on it.
  - We also chain our `"question"` into our `retriever`! This is what ultimately collects the context through Qdrant.
2. We assign our collected context to a `RunnablePassthrough()` from the previous object. This is going to let us simply pass it through to the next step, but still allow us to run that section of the chain.
3. We finally collect our response by chaining our prompt, which expects both a `"question"` and `"context"`, into our `llm`. We also, collect the `"context"` again so we can output it in the final response object.

The key thing to keep in mind here is that we need to pass our context through *after* we've retrieved it - to populate the object in a way that doesn't require us to call it or try and use it for something else.

In [41]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | openai_chat_model, "context": itemgetter("context")}
)

Let's get a visual understanding of our chain!

In [42]:
!pip install -qU grandalf

In [43]:
print(retrieval_augmented_qa_chain.get_graph().draw_ascii())
print(retrieval_augmented_qa_chain.get_graph().draw_mermaid())

                       +---------------------------------+                         
                       | Parallel<context,question>Input |                         
                       +---------------------------------+                         
                           *****                   ****                            
                        ***                            ****                        
                     ***                                   ****                    
+--------------------------------+                             **                  
| Lambda(itemgetter('question')) |                              *                  
+--------------------------------+                              *                  
                 *                                              *                  
                 *                                              *                  
                 *                                              *           

In [44]:
# Mermaid diagramming does not appear to work yet...
print(retrieval_augmented_qa_chain.get_graph().draw_mermaid_png())

ValueError: Failed to render the graph using the Mermaid.INK API. Status code: 400.

Let's try another visual representation:

![image](https://i.imgur.com/Ad31AhL.png)

In [ ]:
# retrieval_augmented_qa_chain.get_graph().draw_mermaid_png()

Let's test our chain out!

In [ ]:
response = retrieval_augmented_qa_chain.invoke({"question" : "What is the most important thing about the iPhone?"})

In [ ]:
response["response"].content

In [ ]:
for context in response["context"]:
  print("Context:")
  print(context)
  print("----")

Let's see if it can handle a query that is totally unrelated to the source documents.

In [ ]:
response = retrieval_augmented_qa_chain.invoke({"question" : "What is the airspeed velocity of an unladen swallow?"})

In [ ]:
response["response"].content

#### ❓ Question #4:

What key innovations did the iPhone introduce?

> HINT: Use your RAG Chain to answer this question.


##### ANSWER:  The key innovations the iPhone introduced include:

1. **Widescreen iPod with Touch Controls**: The iPhone combined the functionality of an iPod with a widescreen and touch controls, allowing users to interact with their music in a new way.\n   \n2. **Revolutionary Mobile Phone**: The iPhone aimed to reinvent the phone by making it easier to make calls, sync contacts from a PC or Mac, and use those contacts more efficiently.
3. **Breakthrough Internet Communications Device**: The iPhone also served as an advanced internet communicator.
4. **Multi-Touch Screen**: The iPhone featured a multi-touch screen, a first in mobile devices.
5. **Miniaturization and Custom Silicon**: It showcased advanced miniaturization and the use of custom silicon.
6. **Tremendous Power Management**: The iPhone had advanced power management capabilities.
7. **OSX Inside a Mobile Device**: The iPhone incorporated OSX, bringing desktop-class applications to a mobile device.
8. **Featherweight Precision Enclosures and Advanced Sensors**: It featured lightweight, precise enclosures and included three advanced sensors.
9. **Desktop Class Applications and Widescreen Video iPod**: The iPhone supported desktop-class applications and acted as a widescreen video iPod.
10. **Over 200 Patents**: Apple filed for over 200 patents for the various inventions included in the iPhone

These innovations collectively positioned the iPhone as a groundbreaking device that combined multiple functionalities into a single, compact form factor.

In [ ]:
response = retrieval_augmented_qa_chain.invoke({"question" : "What key innovations did the iPhone introduce?"})

In [ ]:
response["response"].content